In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [20]:
spark = SparkSession.builder.appName('World Tours').getOrCreate()

In [21]:
df_input = spark.read.options(header='True', InferSchema='True').csv('travel_history.csv')
#df_input.show()

In [22]:
##using dense_rank to get the first and last rank based on traveller and date
df_input.createOrReplaceTempView('fb')
df_start = spark.sql('select traveler,start_city,end_city,date,rnk from (select traveler,start_city,end_city,date, DENSE_RANK() over(partition by traveler order by date) as rnk from fb) where rnk =1')
df_end = spark.sql('select traveler,start_city,end_city,date,rnk from (select traveler,start_city,end_city,date, DENSE_RANK() over(partition by traveler order by date desc) as rnk from fb) where rnk=1')
#df_start.show(20)
#df_end.show(20)

In [23]:
##merging both dataframe w.r.t traveller,start_place,end_place w.r.t date
df_start = df_start.withColumnRenamed("traveler","traveller")
df_start = df_start.withColumnRenamed("start_city","starting_city")
df_start = df_start.withColumnRenamed("end_city","ending_city")
df_start = df_start.withColumnRenamed("date","starting_travel_date")
df_end = df_end.withColumnRenamed("date","ending_travel_date")
df_output = df_start.join(df_end, df_start.traveller == df_end.traveler, 'inner').selectExpr('traveller','starting_city','end_city','starting_travel_date','ending_travel_date')
#df_output.show()

In [24]:
##filtering out traveller who did reached home destination
df_output = df_output.filter('starting_city == end_city').select(col('traveller'))
df_output.show()

+---------+
|traveller|
+---------+
|     Alan|
|     Carl|
|     John|
+---------+

